<a href="https://colab.research.google.com/github/kusanorootbeer/tf2_nb_traingin/blob/master/tf2train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Google drive mount

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


# tensorflow v2.1 install

In [0]:
!pip install tensorflow==2.1.0

# get dataset

In [0]:
!wget http://archive.ics.uci.edu/ml/machine-learning-databases/00341/HAPT%20Data%20Set.zip .
!mkdir drive/My\ Drive/Dataset
!unzip 'HAPT Data Set.zip' -d drive/My\ Drive/Dataset
!rm -rf  HAPT\ Data\ Set.zip

--2020-01-24 11:40:13--  http://archive.ics.uci.edu/ml/machine-learning-databases/00341/HAPT%20Data%20Set.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 79596192 (76M) [application/x-httpd-php]
Saving to: ‘HAPT Data Set.zip’

HAPT Data Set.zip   100%[===================>]  75.91M  10.6MB/s    in 8.1s    

2020-01-24 11:40:21 (9.32 MB/s) - ‘HAPT Data Set.zip’ saved [79596192/79596192]

--2020-01-24 11:40:21--  http://./
Resolving . (.)... failed: No address associated with hostname.
wget: unable to resolve host address ‘.’
FINISHED --2020-01-24 11:40:21--
Total wall clock time: 8.3s
Downloaded: 1 files, 76M in 8.1s (9.32 MB/s)
Archive:  HAPT Data Set.zip
  inflating: drive/My Drive/Dataset/features_info.txt  
  inflating: drive/My Drive/Dataset/README.txt  
  inflating: drive/My Drive/Dataset/RawData/acc_exp01_user01.t

# model building

## import

In [1]:
%tensorflow_version 2.x
import tensorflow as tf
tf.__version__

TensorFlow 2.x selected.


'2.1.0-rc1'

In [0]:
import tensorflow.keras as k
import numpy as np
import pandas as pd
import os 

## Data load

In [0]:
path = "/content/drive/My Drive/Dataset"
features_name = pd.read_csv("{}/features.txt".format(path), header=None)
activity_name = pd.read_csv("{}/activity_labels.txt".format(path), header=None)
train_x = np.loadtxt("{}/Train/X_train.txt".format(path))
train_y = np.loadtxt("{}/Train/y_train.txt".format(path)) -1
train_subject = np.loadtxt("{}/Train/subject_id_train.txt".format(path))
test_x = np.loadtxt("{}/Test/X_test.txt".format(path))
test_y = np.loadtxt("{}/Test/y_test.txt".format(path)) -1
test_subject = np.loadtxt("{}/Test/subject_id_test.txt".format(path))

In [0]:
train_ds = tf.data.Dataset.from_tensor_slices((train_x, train_y)).shuffle(10000).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((test_x, test_y)).batch(32)

## create model

In [0]:
class MLP_Classifier(tf.keras.Model):
  def __init__(self):
    super(MLP_Classifier, self).__init__()
    self.fc1 = k.layers.Dense(300, activation="relu")
    self.fc2 = k.layers.Dense(100, activation="relu")
    self.fc3 = k.layers.Dense(50, activation="relu")
    self.out_put = k.layers.Dense(12, activation="softmax")

  def call(self, x):
    x = self.fc1(x)
    x = self.fc2(x)
    x = self.fc3(x)
    return self.out_put(x)

model = MLP_Classifier()

In [0]:
loss_object = k.losses.SparseCategoricalCrossentropy()
optimizer = k.optimizers.Adam()

In [0]:
train_loss = k.metrics.Mean(name='train_loss')
train_accuracy = k.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = k.metrics.Mean(name='test_loss')
test_accuracy = k.metrics.SparseCategoricalAccuracy(name='test_accuracy')

train_C_matrix = tf.math.confusion_matrix
test_C_matrix = tf.math.confusion_matrix

In [0]:
@tf.function
def train_step(features, labels):
  with tf.GradientTape() as tape:
    predictions = model(features)
    loss = loss_object(labels, predictions)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

  train_loss(loss)
  train_accuracy(labels, predictions)
  return train_C_matrix(labels, tf.math.argmax(predictions, axis=1), name="train_matrix")

In [0]:
@tf.function
def test_step(features, labels):
  predictions = model(features)
  t_loss = loss_object(labels, predictions)

  test_loss(t_loss)
  test_accuracy(labels, predictions)
  return test_C_matrix(labels, tf.math.argmax(predictions, axis=1), name="test_matrix")

In [0]:
@tf.function
def eval_step(features, labels):
  prediciont = model(features)
  

In [11]:
EPOCHS = 20

for epoch in range(EPOCHS):
  for features, labels in train_ds:
    train_mat =  train_step(features, labels)

  for test_features, test_labels in test_ds:
    test_mat = test_step(test_features, test_labels)

  template = 'Epoch {}, Loss: {:.7}, Accuracy: {:.7}, Test Loss: {:.7}, Test Accuracy: {:.7}'
  print (template.format(epoch+1, train_loss.result(), train_accuracy.result()*100, test_loss.result(), test_accuracy.result()*100))
  # print(test_mat)
  # print(train_mat)
  # print(predictions)
  
  # reset metrics
  train_loss.reset_states()
  train_accuracy.reset_states()
  test_loss.reset_states()
  test_accuracy.reset_states()


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

Epoch 1, Loss: 0.5066722, Accuracy: 81.5244, Test Loss: 0.2924864, Test Accuracy: 87.57116
Epoch 2, Loss: 0.2038591, Accuracy: 92.4295, Test Loss: 0.2568375, Test Accuracy: 90.41746
Epoch 3, Loss: 0.1412918, Accuracy: 94.39938, Test Loss: 0.2595185, Test Accuracy: 90.3542
Epoch 4, Loss: 0.1259761, Accuracy: 95.03025, Test Loss: 0.1855275, Test Accuracy: 93.04238
Epoch 5, Loss: 0.132892, Accuracy: 94.51526, Test Loss: 0.2116889, Test Accuracy: 91.42948
Epoch 6, Loss: 0.09901819, Accuracy: 96.12463, Test 